In [7]:
import json
from pprint import pprint
orig_train = json.loads(open("train.json").read())
orig_test = json.loads(open("test.json").read())

print "# records in train:",len(orig_train)
print "# records in test:",len(orig_test)

orig_train_text = []
orig_train_labels = []
kaggle_test_text = []
kaggle_test_id = []
#TODO: create more features arrays here

for record in orig_train:
    orig_train_text.append(record['request_text'])
    if record['requester_received_pizza']:
        orig_train_labels.append(1)
    else:
        orig_train_labels.append(0)

print len(orig_train_labels)
print sum(orig_train_labels)
print orig_train_labels[:50]

for record in orig_test:
    kaggle_test_text.append(record['request_text_edit_aware'])
    kaggle_test_id.append(record['request_id'])
    #TODO: build more features arrays here

#pprint(test[1])
#print orig_train_labels[:10]
#print orig_train_text[0]
#print kaggle_test_text[1]

# Break data up into training and testing data
train_text = orig_train_text[:3500]
train_labels = orig_train_labels[:3500]

dev_text = orig_train_text[3501:]
dev_labels = orig_train_labels[3501:]


def write_solution_to_file(prediction, filename):
    if len(prediction) == len(kaggle_test_id):
        print "Writing to file", filename
        outfile = open(filename,'w')
        outfile.write("request_id,requester_received_pizza\n")
        for i in range(0,len(prediction)):
            outfile.write(kaggle_test_id[i]+','+str(prediction[i])+'\n')
        return 1
    else:
        print "Prediction dimension mismatch"
        return 0


# records in train: 4040
# records in test: 1631
4040
994
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0]


In [5]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV
from sklearn.feature_extraction.text import *
from sklearn import metrics

cv_train = CountVectorizer()
dtm_train = cv_train.fit_transform(train_text)
cv_dev = CountVectorizer(vocabulary=cv_train.get_feature_names())
dtm_dev = cv_dev.fit_transform(dev_text)

mnb = MultinomialNB()
mnb_parameters = {'alpha':[.1,.001,.0001,.00001]}
mnbgs = GridSearchCV(mnb, mnb_parameters)
mnbgs.fit(dtm_train, train_labels)
mnb_pred = mnbgs.predict(dtm_dev)
best_params = mnbgs.best_params_

print "\nMNB f1 score: ", metrics.f1_score(dev_labels, mnb_pred, average='binary')
print "MNB best params:", best_params

# Now that we have scanned some parameters, we can proceed with the best one but train with all data
cv_all = CountVectorizer()
dtm_all = cv_all.fit_transform(orig_train_text)
cv_kaggle = CountVectorizer(vocabulary=cv_all.get_feature_names())
dtm_kaggle = cv_kaggle.fit_transform(kaggle_test_text)

mnb_kaggle = MultinomialNB(alpha=best_params['alpha'])
mnb_kaggle.fit(dtm_all, orig_train_labels)
mnb_pred = mnb_kaggle.predict(dtm_kaggle)

write_solution_to_file(mnb_pred, "test-submission.csv")


MNB f1 score:  0.286995515695
MNB best params: {'alpha': 1e-05}
Writing to file test-submission.csv


1